In [1]:
def isString(data):
    try:
        float(data)
        return False
    except:
        pass
        return True
def gini_index(groups, classes):
    total_rows = 0.0;
    for grp in groups:
        total_rows += len(grp)
        
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        for val in classes:
            count = 0
            for row in group:
                if(row[-1] == val):
                    count = count+1;
            proportion = count/size   
            score += proportion * proportion
        gini += (1.0 - score) * (size / total_rows)
    return gini
def splitData(data, value, ind):
    left =[]
    right = []
    for row in data:
        if(row[ind] >= value):
            right.append(row)
        else:
            left.append(row)
    return np.array(left), np.array(right)
def evalSplitForest(data, num_features):
    classes = np.unique(data[:,-1])
    split_Ind, split_Val, Split_Data, minscore = 9999999, 9999999, None, 9999999
    sample_features = []
    d ={}
    while len(sample_features) < num_features:
        ind = random.randrange(len(data[0])-1)
        if ind not in sample_features:
            sample_features.append(ind)
        
    for col in sample_features:
        for row in data:
            samples =[]
            left , right = splitData(data, row[ind], ind)
            samples.append(left)
            samples.append(right)
            giniScore = gini_index(samples, classes)
            if(giniScore<minscore):
                split_Ind, split_Val, Split_Data, minscore = ind, row[ind], samples, giniScore
    d['index'] = split_Ind
    d['val'] = split_Val
    d['sample'] = Split_Data
    d['score'] = minscore
    return d

In [2]:
def splitNodeForest(node , num_features):
    left, right = node['sample']
    
    if((left.size==0) or (right.size==0)):
        classes = []
        if(left.size==0):
            classes = right[:,-1]  
        else:
            classes = left[:,-1]
        labelCount  = collections.Counter(classes)
        top = labelCount.most_common()
        node['left'] = node['right'] = top[0][0]
        return
    
    if (len(left)<=1):
        classes = left[:,-1]
        labelCount  = collections.Counter(classes)
        top = labelCount.most_common()
        node['left'] = top[0][0]
    else:
        node['left'] = evalSplitForest(left, num_features)
        splitNodeForest(node['left'],num_features)
    
    if(len(right)<=1):
        classes = right[:,-1]
        labelCount  = collections.Counter(classes)
        top = labelCount.most_common()
        node['right'] = top[0][0]
    else:
        node['right'] = evalSplitForest(right, num_features)
        splitNodeForest(node['right'],num_features)

In [3]:
def predict(node, row):
    if row[node['index']] < node['val']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']
        

In [4]:
def metrics(predictedLabels,trueLabels):
    a=0
    b=0
    c=0
    d=0
    for i, j in zip(trueLabels,predictedLabels):
            if ((j == 1) & (i == j)):
                a += 1
            elif ((j == 0) & (i != j)):
                b += 1
            elif ((j == 1) & (i != j)):
                c += 1
            elif ((j == 0) & (i == j)):
                d += 1
    sum = a+b+c+d
    if sum == 0:
        accuracy = "Not possible to compute"
    else:
        accuracy = (a+d)/sum
    if a+c == 0:
        precision = "Not possible to compute"
    else:
        precision = a/(a+c)
    if a+b == 0:
        recall = "Not possible to compute"
    else:
        recall = a/(a+b)
    if a+b+c == 0:
        f1 = "Not possible to compute"
    else:
        f1 = 2*a/(2*a+b+c)
    return accuracy,precision,recall,f1

In [5]:
import random as random
def randomForest(traindata, testdata, num_trees, num_features ):
    forest_List = []
    predicted_List = []
    sample_size = int(len(traindata)/num_trees)
    for i in range(0, num_trees):
        sample = []
        while(len(sample)<sample_size):
            i = random.randrange(len(traindata))
            sample.append(traindata[i])
        sample = np.array(sample)
        root = evalSplitForest(sample, num_features)
        splitNodeForest(root, num_features)
        forest_List.append(root)

    for row in testdata:
        row_prediction = []
        for tree in forest_List:
            row_prediction.append(predict(tree, row))
        countDict = collections.Counter(row_prediction)
        predicted_List.append(countDict.most_common()[0][0])

    actual_List = list(testdata[:,-1])
    return actual_List, predicted_List

In [6]:
import numpy as np
import collections
import sys
K = 10
filename = input("enter file name with relative path: ")
data = []
for i in open(filename, 'r').readlines():
        data.append(i.rstrip().split())    
data = np.array(data);
num_trees = int(input("Enter no of trees: "))
num_features = int(input("Enter no of features: "))
randomdata = np.random.permutation(len(data))
length = len(randomdata)
Allaccuracies = []
Allprecision = []
Allrecall =[]
Allf1score = []
length = len(randomdata)
for i in range(K):
    tdata = []
    tdataLabels = []
    vdata = []
    vdataLabels = []
    start = i *length//10
    end = (i+1)*length//10
    for j in range(length):
        if(j in range(start,end)):
            vdata.append(data[randomdata[j]])
            vdataLabels.append(data[randomdata[j]][-1])
        else:
            tdata.append(data[randomdata[j]])
            tdataLabels.append(data[randomdata[j]][-1])
    tdata = np.array(tdata)
    vdata = np.array(vdata)
#     print(tdata.shape)
    actualLabels, predictedLabels  = randomForest(tdata,vdata, num_trees, num_features)
    actualLabels = np.array(actualLabels).astype(int)
    predictedLabels = np.array(predictedLabels).astype(int)
    accuracy,precision,recall,f1 = metrics(predictedLabels,actualLabels)
    if not (isString(accuracy)):
        Allaccuracies.append(accuracy)
    if not (isString(precision)):
        Allprecision.append(precision)
    if not (isString(recall)):
        Allrecall.append(recall)
    if not (isString(f1)):
        Allf1score.append(f1)
    print("\nIteration " + str(i))
    print("Training data Length: " + str(len(tdata)))
    print("Validation data Length: " + str(len(vdata)))
    print("accuracy " + str(accuracy))
    print("precision " + str(precision))
    print("recall " + str(recall))
    print("f1 " + str(f1))
print("\nAverageaccuracy " + str(np.array(Allaccuracies).mean()))
print("Averageprecision " + str(np.array(Allprecision).mean()))
print("Averagerecall " + str(np.array(Allrecall).mean()))
print("Averagef1 " + str(np.array(Allf1score).mean()))

Enter no of trees: 5
Enter no of features: 10

Iteration 0
Training data Length: 513
Validation data Length: 56
accuracy 0.9107142857142857
precision 0.8636363636363636
recall 0.9047619047619048
f1 0.8837209302325582

Iteration 1
Training data Length: 512
Validation data Length: 57
accuracy 0.8245614035087719
precision 0.782608695652174
recall 0.782608695652174
f1 0.782608695652174

Iteration 2
Training data Length: 512
Validation data Length: 57
accuracy 0.9824561403508771
precision 1.0
recall 0.9473684210526315
f1 0.972972972972973

Iteration 3
Training data Length: 512
Validation data Length: 57
accuracy 0.9649122807017544
precision 0.9375
recall 0.9375
f1 0.9375

Iteration 4
Training data Length: 512
Validation data Length: 57
accuracy 0.9122807017543859
precision 0.9411764705882353
recall 0.8
f1 0.8648648648648649

Iteration 5
Training data Length: 512
Validation data Length: 57
accuracy 0.9473684210526315
precision 0.9523809523809523
recall 0.9090909090909091
f1 0.930232558139534

In [ ]:
# import csv
# import numpy as np
# import collections
# def loadCSV(filename): 
#     ''' 
#     function to load dataset 
#     '''
#     with open(filename,"r") as csvfile: 
#         lines = csv.reader(csvfile) 
#         dataset = list(lines) 
#         for i in range(len(dataset)): 
#             dataset[i] = [float(x) for x in dataset[i]]      
#     return np.array(dataset)
# data = loadCSV(r'C:\Users\sampr\Documents\dataMining\Project3\cse-601-project-3-fall-2019\train_features.csv')
# data1 = loadCSV(r'C:\Users\sampr\Documents\dataMining\Project3\cse-601-project-3-fall-2019\test_features.csv')
# actualLabels, predictedLabels  = randomForest(data,data1, 5, 10)

In [ ]:
# s.to_csv("kaggle1.csv", sep=',', encoding='utf-8')